# 🧬 Geometry-Complete Equivariant Diffusion
## De Novo Drug Design Training

**Data**: Pitt.edu direct wget

## Cell 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None!"}')

%pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm scikit-learn
print('✅ Setup complete')

## Cell 2: Clone Repo

In [ ]:
import os
REPO = '/content/drive/MyDrive/geom_diffusion'
if not os.path.exists(REPO):
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {REPO}
%cd {REPO}
!git pull origin master

## Cell 3: Download CrossDocked

In [ ]:
import os

DATA_DIR = '/content/data/raw'
os.makedirs(DATA_DIR, exist_ok=True)

existing = [d for d in os.listdir(DATA_DIR) if os.path.isdir(f'{DATA_DIR}/{d}')]

if len(existing) < 10:
    print('📥 Downloading downsampled CrossDocked2020 (~5GB)...')
    !wget -q --show-progress -O /content/crossdocked.tgz \
        https://bits.csb.pitt.edu/files/crossdock2020/downsampled_CrossDocked2020_v1.3.tgz
    
    print('\n📦 Extracting...')
    !tar -xzf /content/crossdocked.tgz -C {DATA_DIR}/
    !rm /content/crossdocked.tgz
    print('✅ Done!')
else:
    print(f'✅ Data exists: {len(existing)} folders')

folders = [d for d in os.listdir(DATA_DIR) if os.path.isdir(f'{DATA_DIR}/{d}')]
print(f'📁 Total pocket folders: {len(folders)}')

## Cell 4: Analyze Size Distribution

In [ ]:
import os
import numpy as np
from pathlib import Path
from tqdm import tqdm

DATA_DIR = '/content/data/raw'
pocket_dirs = [Path(DATA_DIR) / d for d in os.listdir(DATA_DIR) if os.path.isdir(f'{DATA_DIR}/{d}')]

# Sample 500 pockets to understand distribution
sizes = []
for pocket_dir in tqdm(pocket_dirs[:500], desc='Analyzing'):
    all_pdb = list(pocket_dir.glob('*.pdb'))
    if all_pdb:
        try:
            with open(all_pdb[0], 'r') as f:
                atom_count = sum(1 for line in f if line.startswith('ATOM'))
            sizes.append(atom_count)
        except:
            pass

sizes = np.array(sizes)
print(f'\n📊 Pocket size distribution (atoms):')
print(f'   Min: {sizes.min()}, Max: {sizes.max()}')
print(f'   Mean: {sizes.mean():.0f}, Median: {np.median(sizes):.0f}')
print(f'   Percentiles: 25%={np.percentile(sizes,25):.0f}, 50%={np.percentile(sizes,50):.0f}, 75%={np.percentile(sizes,75):.0f}')

# Suggest max size
p75 = np.percentile(sizes, 75)
print(f'\n💡 Recommended max size: {int(p75)} atoms (75th percentile)')

## Cell 5: Create 5K Subset (Adjusted Filters)

In [ ]:
import os
import pickle
import numpy as np
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

np.random.seed(42)

DATA_DIR = '/content/data/raw'
OUT_DIR = '/content/data/crossdocked'
os.makedirs(OUT_DIR, exist_ok=True)

# ADJUSTED FILTERS for this dataset
MAX_SIZE = 800  # Increased! Most pockets are larger
MIN_LIGANDS = 3  # Relaxed - avg is 7

print(f'⚙️ Filters: max_size={MAX_SIZE}, min_ligands={MIN_LIGANDS}')

pocket_dirs = [Path(DATA_DIR) / d for d in os.listdir(DATA_DIR) if os.path.isdir(f'{DATA_DIR}/{d}')]
print(f'Found {len(pocket_dirs)} pocket directories')

pockets = defaultdict(list)
pocket_info = {}

for pocket_dir in tqdm(pocket_dirs, desc='Processing'):
    pocket_id = pocket_dir.name
    all_pdb = list(pocket_dir.glob('*.pdb'))
    all_sdf = list(pocket_dir.glob('*.sdf'))
    
    pocket_pdb = [p for p in all_pdb if 'rec' in p.name.lower() or 'pocket' in p.name.lower()]
    if not pocket_pdb:
        pocket_pdb = all_pdb[:1]
    
    if pocket_pdb and all_sdf:
        try:
            with open(pocket_pdb[0], 'r') as f:
                atom_count = sum(1 for line in f if line.startswith('ATOM'))
        except:
            atom_count = 500
        
        pocket_info[pocket_id] = {'size': atom_count, 'pdb': str(pocket_pdb[0])}
        
        for lig in all_sdf:
            pockets[pocket_id].append({
                'pocket_pdb': str(pocket_pdb[0]),
                'ligand_sdf': str(lig),
                'pocket_id': pocket_id,
                'num_atoms': atom_count
            })

print(f'\n✅ Processed {len(pockets)} pockets')

# Apply filters
valid = [p for p, samples in pockets.items() 
         if pocket_info[p]['size'] <= MAX_SIZE and len(samples) >= MIN_LIGANDS]
print(f'After filter: {len(valid)} pockets')

# Stratify by size (adjusted bins)
sizes = [pocket_info[p]['size'] for p in valid]
p33, p66 = np.percentile(sizes, [33, 66])
print(f'Size terciles: small<{p33:.0f}, medium<{p66:.0f}, large>={p66:.0f}')

small = [p for p in valid if pocket_info[p]['size'] <= p33]
medium = [p for p in valid if p33 < pocket_info[p]['size'] <= p66]
large = [p for p in valid if pocket_info[p]['size'] > p66]

print(f'Bins: Small={len(small)}, Medium={len(medium)}, Large={len(large)}')

# Select from each bin
np.random.shuffle(small)
np.random.shuffle(medium)
np.random.shuffle(large)

n_small = min(40, len(small))
n_medium = min(40, len(medium))
n_large = min(40, len(large))
selected = small[:n_small] + medium[:n_medium] + large[:n_large]

print(f'\nSelected: {len(selected)} pockets')

# Split
np.random.shuffle(selected)
split_idx = int(len(selected) * 0.83)
train_pockets = selected[:split_idx]
val_pockets = selected[split_idx:]

print(f'Train: {len(train_pockets)}, Val: {len(val_pockets)}')

# Create datasets
train_samples = [s for p in train_pockets for s in pockets[p][:50]]
val_samples = [s for p in val_pockets for s in pockets[p][:50]]

print(f'📊 Train: {len(train_samples)}, Val: {len(val_samples)}')

# Save
with open(f'{OUT_DIR}/train_data.pkl', 'wb') as f:
    pickle.dump(train_samples, f)
with open(f'{OUT_DIR}/val_data.pkl', 'wb') as f:
    pickle.dump(val_samples, f)

print('💾 Saved!')

## Cell 6: Update Config

In [ ]:
import yaml

%cd /content/drive/MyDrive/geom_diffusion

with open('configs/debug_t4.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['root'] = '/content/data'
cfg['data']['train_file'] = 'crossdocked/train_data.pkl'
cfg['data']['val_file'] = 'crossdocked/val_data.pkl'
cfg['training']['max_epochs'] = 50
cfg['training']['batch_size'] = 2  # Reduced for larger pockets
cfg['hardware']['num_workers'] = 2

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(cfg, f)

print('✅ Config updated (batch_size=2 for larger pockets)')

## Cell 7: Train 🚀

In [ ]:
import shutil
from pathlib import Path

%cd /content/drive/MyDrive/geom_diffusion

for cache in ['/content/data/cache', 'data/cache']:
    if Path(cache).exists():
        shutil.rmtree(cache)

!python train.py --config configs/debug_t4.yaml --checkpoint_dir checkpoints